# Aerospike 'Scale Warriors' Session sponsored by AWS

## Purpose of session

* Use Aerospike in context of Jupyter notebooks - supports learning / prototyping
* Learn about Aerospike Expressions
* Investigate a real world use case - fraud detection
* Gain an understanding of a machine learning led process supported by Aerospike

## Fraud Model: 
* Track all customer accounts.
* For any specific customer account:
 *  Maintain average transaction amount for each payee.
 *  Flag transactions for a given account that appear fraudulent (by proprietary criteria)
 *  Have the ability to tune the fraud detection criteria.

<img src="./graphics/fraud_detection_1.png"
     alt="Fraud Detection"
     style="center; margin-right: 10px;"
     width="800"
     height="640"/>

## Presentation Outline 

* Review of the Jupyter Notebook Environment
* Interactive Java Development Environment
* Connecting to Aerospike
* Simple Fraud Detection Problem Description
* Solution Development



# Jupyter Notebook Overview
What is happening behind the scenes...

![fig2](./graphics/JupNb_Overview.png)

We are using the iJava (Interactive Java Kernel for Jupyter Notebook) in this example.

A kernel can be thought of as a language specific computational engine.

##### Jupyter Notebook - Cells and Execution / Edit shortcuts 

We have a mix of Markdown cells (text description in Markdown) and Code cells (we have chosen iJava Kernel).
<img src="./graphics/JupNb_Cells.png"
     alt="Fraud Detection"
     style="center; margin-right: 10px;"
     width="250"
     height="400"/>


You can use the menu above to change a cell from being a Code cell to a Markdown cell.

**Cntrl-Enter** executes a cell. Try it.

Clicking anywhere in text display (Markdown) cell and hitting **Enter**, puts you in the cell edit mode. Try it.

Code cells can be edited directly. To run an edited code cell, just hit **Cntrl-Enter**.

You can add a new blank cell, cells can be deleted, copied, pasted or moved up or down using the menu icons as shown below:

![fig4](./graphics/JupNb_CellEditing.png)

### Housekeeping - Wipe out any prior records on the Aerospike Server

We have a namespace **_test_** pre-defined on the server. Lets truncate it using aql.

This is needed while doing code development. If you want to clear the iJava Kernel of all Java objects and run all cells from scratch, Kernel->Restart & Run All, this will ensure any records written on the underlying Aerospike cluster are purged.

First, we need required imports for using %sh in interactive Java Kernel. (This is specific to the iJava Kernel implementation by **Spencer Park** that we are using.)

In [25]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

### Runing aql in iJava
We can run aql commands in a file. No output will be displayed. 
In _trucate_test.aql_, we have a single AQL command: TRUNCATE test



In [26]:
%sh aql -h 127.0.0.1 -f "./aqlScripts/truncate_test.aql"

### Runing aql via Terminal Tab

Alternatively, you can also open a separate terminal window (TAB) from the Home Page and run aql interactively in the Terminal.  Please **DO NOT CLICK ON QUIT** on Login Page. It will kill the Jupyter Notebook Server.

![fig5](./graphics/JupNb_Terminal.png)

### Get our Aerospike Java Client talking to the Single Node Aerospike Server 

We will be discussing the data model developmnet interactively. We will code as we progress through the discussion. So let us connect our client application's interactive coding environment with the single node Aerospike server. This will get our real time interaction started.

As we write code in following Jupyter Notebook cells, it will be executed on the Single node Aerospike Server. 

#### Add Java Client POM Dependency  
Jupyter Notebook way!

In [27]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client</artifactId>
    <version>6.0.0</version>
  </dependency>
</dependencies>

####  Add required Java Client Imports

These are some of the Aerospike Java Client imports needed to start developing our Application interactively.  We will add others, as needed, as we develop our solution.

In [28]:
//Require Imports
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Record;
import com.aerospike.client.Value;
System.out.println("Client modules imported.");

Client modules imported.


#### Connect to the Aerospike Server
Instantiate the client object. Let us write a record and read it back.
We have a namespace **_test_** pre-defined on the server.

In [29]:
AerospikeClient client = new AerospikeClient("127.0.0.1", 3000);
System.out.println("Initialized the client and connected to the cluster.");

Key key = new Key("test", "testset", "key1");
System.out.println("Working with record key:");
System.out.println(key);  //Will show the 20 byte digest

Initialized the client and connected to the cluster.
Working with record key:
test:testset:key1:bf6c1d13e7cd10c5bd022d27e7df170c0bccd6e1


#### Test Record Data Model

![fig_DM_1](./graphics/Fraud_DM_1.png)

In [30]:
//Insert a record with two bins, b1 with string v1 and b2 with integer 2 as data.

WritePolicy wPolicy = new WritePolicy();

Bin b1 = new Bin("b1", Value.get("val1"));
Bin b2 = new Bin("b2", Value.get(2));

client.put(wPolicy, key, b1, b2);

##### About Jupyter Notebook Cell Execution sequence

If a previous code cell was executed, objects instantiated in it are available in the next cell that you execute. 

You can execute individual cells in any order - this is interactive execution.

Order of execution will determine the net result.  i.e. you can go back and forth between the record insertion cell above, and record read cell below. 

![fig_exec](./graphics/JupNb_Execution.png)

If you edit the insertion values, you must re-execute the write cell to put the new value on the Aerospike Server. Then, if you execute the read cell, you will see the updated value.

**_If you have lost track about where you are in the sequence, go to the top Menu, under Cell, Choose "Run All" to run all cells - top to bottom - in this notebook's sequence._** If you want to clear all objects in the kernel and restart from a fresh state, use **Kernel -> Restart & Run All**.


In [31]:
//Read the record that we just inserted

Record record = client.get(null, key);
System.out.println("Read back the record.");

System.out.println("Record values are:");
System.out.println(record);

Read back the record.
Record values are:
(gen:1),(exp:392909319),(bins:(b1:val1),(b2:2))


## Getting Back to Fraud Detection
Now that we are all setup, let us get back to discussing our Fraud Detection Example.

## Simple Incoming Transaction

* New Account:

 *  acct_id:  _**override**_, _**withdrawl_amount**_, _**payee**_, current_balance, user_profile_json 
 
* New Transaction to existing account:

 *  acct_id:  _**override**_, _**withdrawl_amount**_, _**payee**_ 


We can use the DocumentAPI to write to the user_profile_json bin. 

However, for this hands-on, we will focus on the fraud detection part. For simplicity, we will just enter the username as a string for _user_profile_json_.  

If _override_ bin is set _true_, do not flag as fraudulent. We will use override true to insert our training data as well as allow the application to enter a genuine new payee.



## Record Data Model
While Aerospike limits bin names to 15 characters, shorter the better.

### Fraud Detection - Proposed Record Data Model:
* acct_id (Primary Key):
 *  ovr (override bin, boolean)
 *  lastamnt (last successful withdrawl_amount bin, integer, cents) 
 *  payee (payee bin, string)  
 *  bal (current_balance bin, integer, cents) 
 *  profile (user_profile_json bin, string, username) 
 
 ![fig_DM_2](./graphics/Fraud_DM_2.png)


#### New  _acct_id_ record creation

**Exercise** Write code to insert one record in namespace: **test**, setname: **accts**, key = **1** integer value (int acct_id=1). 
Add the bins described above with ovr=true, lastamnt=-10000, payee="SELF", bal=10000. 

(We will update new acct creation later with _automatic calculation of balance_.)


In [32]:
// New acct_id record creation
// Lets create a test record with this data structure

int iAcct_id = 1;  //Acct_id = 1 to 1000.
boolean bOvr = true;
int iWdAmount = -10000;
String sPayee = "SELF";
int iBal = 10000;

Key user = new Key("test", "accts", iAcct_id);  

Bin bOvr = new Bin("ovr", Value.get(bOvr));
Bin bAmnt = new Bin("lastamnt", Value.get(iWdAmount));
Bin bPayee = new Bin("payee", Value.get(sPayee));
Bin bBal = new Bin("bal", Value.get(iBal));
Bin bProfile = new Bin("profile", Value.get("user"+iAcct_id));

WritePolicy wPolicy = new WritePolicy();
client.put(wPolicy, user, bOvr, bAmnt, bPayee, bBal, bProfile);



In [33]:
//Check 
System.out.println( client.get(null, user) );

(gen:1),(exp:392909320),(bins:(lastamnt:-10000),(payee:SELF),(bal:10000),(profile:user1))


#### Update  _acct_id_ record with incoming transaction

Update record with a withdrawl transaction for `$80.00` or integer value 8000 cents. Recall we only have `$100.00`or 10000 cents. We should be left with 2000 cents. 

**Incoming Transaction:**
 *  acct_id:  _**override**_, _**withdrawl_amount**_, _**payee**_ 

How can we do this atomically? --> Use Write Expressions.


## Using OperationExpression write()



## Record Transaction Considerations

In [34]:
import com.aerospike.client.Operation;
import com.aerospike.client.exp.Exp;
import com.aerospike.client.exp.ExpOperation;
import com.aerospike.client.exp.ExpWriteFlags;
import com.aerospike.client.exp.Expression;

int iAcct_id = 1;  //Acct_id = 1 to 1000.
boolean bOvr = false;
int iWdAmount = 8000;
String sPayee = "VISA";

Key user = new Key("test", "accts", iAcct_id); 

Bin bOverride = new Bin("ovr", Value.get(bOvr));
//Later we will use the override boolean value for decisioning.
Bin bAmnt = new Bin("lastamnt",Value.get(iWdAmount));
Bin bPayee = new Bin("payee",Value.get(sPayee));

Expression balExp = Exp.build(Exp.sub(Exp.intBin("bal"),Exp.val(iWdAmount)));

//wPolicy is default here, no filter expression used (yet!)
//Update withdrawl amount in amnt bin
//Expression Operation write for bal bin

WritePolicy wPolicy = new WritePolicy();
record = client.operate( wPolicy, user,   
          Operation.put(bOverride),
          Operation.put(bAmnt),     
          Operation.put(bPayee), 
          // ExpWriteFlags - CREATE_ONLY / UPDATE_ONLY / error handling ...
          ExpOperation.write("bal", balExp, ExpWriteFlags.DEFAULT)
         );


In [35]:
//Check 
// Note profile bin remains unchanged
System.out.println( client.get(null, user) );

(gen:2),(exp:392909320),(bins:(lastamnt:8000),(payee:VISA),(bal:2000),(profile:user1),(ovr:0))


#### (Recap: Record Data Model)

 ![fig_DM_2](./graphics/Fraud_DM_2.png)


### Simple balance check
If **withdrawl_amount**  _is greater than_ **current_balance**, reject the transaction. 

Lets modify our code to add this Filter Expression condition.

In [36]:
import com.aerospike.client.AerospikeException;
import com.aerospike.client.ResultCode;

int iAcct_id = 1;  //Acct_id = 1 to 1000.
boolean bOvr = false;
int iWdAmount = 3000;
String sPayee = "Cash";

Key user = new Key("test", "accts", iAcct_id); 

Bin bOverride = new Bin("ovr", Value.get(bOvr));
Bin bAmnt = new Bin("lastamnt",Value.get(iWdAmount));
Bin bPayee = new Bin("payee",Value.get(sPayee));

WritePolicy wPolicy = new WritePolicy();
// Filter expression is part of the policy object, supplied when calling 'operate'
wPolicy.filterExp = Exp.build(Exp.or(
    Exp.ge(Exp.intBin("bal"),Exp.val(iWdAmount)), 
    Exp.val(bOvr))
    );
// Want it to fail when filterExp = false
wPolicy.failOnFilteredOut = true;

Expression balExp = Exp.build(Exp.sub(Exp.intBin("bal"),Exp.val(iWdAmount)));

try {
//wPolicy now has a filter expression added
//Update withdrawl amount in amnt bin
//Expression Operation write for bal bin

record = client.operate( wPolicy, user,   
          Operation.put(bOverride),
          Operation.put(bAmnt),     
          Operation.put(bPayee),           
          ExpOperation.write("bal", balExp, ExpWriteFlags.DEFAULT)
         );
} catch (AerospikeException e){
  int rc = e.getResultCode();
  if(rc == ResultCode.FILTERED_OUT){
    System.out.println("Insufficient Balance. Transaction Rejected.");
  }
}

Insufficient Balance. Transaction Rejected.


In [37]:
//Check 
System.out.println( client.get(null, user) );

(gen:2),(exp:392909320),(bins:(lastamnt:8000),(payee:VISA),(bal:2000),(profile:user1),(ovr:0))


### Application initiates first transaction to create the account.

To create the account in this first transaction in the account, the transaction is expected to be made with **override** _true_. _withdrawl_amount_ should be a negative number (deposit), starting _current_balance_ on new account creation will be zero, after the deposit, it will _(0 - (-amount)) = amount deposited_.  Add user profile (user name string). For payee name, use SELF.

If Bin "bal" exists, (new balance = old balance - withdrawl_amount), else (0 - withdrawl_amount).

If profile bin exists, do not update it otherwise add profile string in profile bin. (Use ExpWriteFlags)

We are still working this record data model:

![fig_DM_2](./graphics/Fraud_DM_2.png)



In [38]:
import com.aerospike.client.AerospikeException;
import com.aerospike.client.ResultCode;

int iAcct_id = 2;  //Acct_id = 1 to 1000.
boolean bOvr = true;
int iWdAmount = -5000;
String sPayee = "SELF";
String sProfile = "user"+iAcct_id;

Key user = new Key("test", "accts", iAcct_id); 

Bin bOverride = new Bin("ovr", Value.get(bOvr));
Bin bAmnt = new Bin("lastamnt",Value.get(iWdAmount));
Bin bPayee = new Bin("payee",Value.get(sPayee));
Bin bProfile = new Bin("profile", Value.get(sProfile));

// Same as previous filter expression
WritePolicy wPolicy = new WritePolicy();
wPolicy.filterExp = Exp.build(Exp.or(
    Exp.ge(Exp.intBin("bal"),Exp.val(iWdAmount)), 
    Exp.val(bOvr))
    );
wPolicy.failOnFilteredOut = true;

// Expression for calculating balance
Expression balExp = 
    Exp.build(
        // If balance exists
        Exp.cond(Exp.binExists("bal"), 
                 // Subtract the withdrawl amount from it
                 Exp.sub(Exp.intBin("bal"),Exp.val(iWdAmount)),
                 // Else subtract from zero
                 Exp.sub(Exp.val(0), Exp.val(iWdAmount))
        )
    );

// We will use the below to ensure that the profile only gets written once
Expression profileExp = Exp.build(Exp.val(sProfile));
  
// Run in a try/catch block as we may get 'filtered out' exception
try {
    // wPolicy now has a filter expression added - can't go overdrawn unless override is set
    // balExp manages the account balance
    // An expression ensures profile is only set once

    record = client.operate( wPolicy, user,   
              Operation.put(bOverride),
              Operation.put(bAmnt),     
              Operation.put(bPayee),           
              // Balance update
              ExpOperation.write("bal", balExp, ExpWriteFlags.DEFAULT),
              // Write to the profile bin on creation only. 'Fail' silently otherwise
              ExpOperation.write("profile", profileExp, ExpWriteFlags.CREATE_ONLY | ExpWriteFlags.POLICY_NO_FAIL)
    );
} 
catch (AerospikeException e){
  int rc = e.getResultCode();
  if(rc == ResultCode.FILTERED_OUT){
    System.out.println("Insufficient Balance. Transaction Rejected.");
  }
}

In [39]:
// Check - should have a $5000 balance
System.out.println( client.get(null, user) );

(gen:1),(exp:392909321),(bins:(ovr:1),(lastamnt:-5000),(payee:SELF),(bal:5000),(profile:user2))


### If it is a Deposit (Negative Withdrawl Amount):
* Skip Fraud Check
* Update **current_balance** and **withdrawl_amount**, and return.

Previous code should work with **override** = _false_.

Try and check with acct_id = 2. Add -3000 amount

In [46]:
import com.aerospike.client.AerospikeException;
import com.aerospike.client.ResultCode;

int iAcct_id = 2;  //Acct_id = 1 to 1000.
boolean bOvr = false;
int iWdAmount = -3000;
String sPayee = "SELF";
String sProfile = "do_not_update_user"+iAcct_id;  //Check we are not updating the profile string now.

Key user = new Key("test", "accts", iAcct_id); 

Bin bOverride = new Bin("ovr", Value.get(bOvr));
Bin bAmnt = new Bin("lastamnt",Value.get(iWdAmount));
Bin bPayee = new Bin("payee",Value.get(sPayee));
Bin bProfile = new Bin("profile", Value.get(sProfile));

// Re-use wPolicy / balExp / profileExp
try {
    record = client.operate( wPolicy, user,   
              Operation.put(bOverride),
              Operation.put(bAmnt),     
              Operation.put(bPayee),           
              // Balance update
              ExpOperation.write("bal", balExp, ExpWriteFlags.DEFAULT),
              // Write to the profile bin on creation only. 'Fail' silently otherwise
              ExpOperation.write("profile", profileExp, ExpWriteFlags.CREATE_ONLY | ExpWriteFlags.POLICY_NO_FAIL)
    );
} 
catch (AerospikeException e){
  int rc = e.getResultCode();
  if(rc == ResultCode.FILTERED_OUT){
    System.out.println("Insufficient Balance. Transaction Rejected.");
  }
}

In [47]:
//Check - should have an $8000 balance
System.out.println( client.get(null, user) );

(gen:5),(exp:392909546),(bins:(ovr:0),(lastamnt:-3000),(payee:SELF),(bal:17000),(profile:user2))


### Should it be flagged as "Potential_Fraud"?
We will implement a simple **Go/NoGo** Model - we are not computing a fraud score in our hands-on exercise today.
For each **_acct_id_** : 



#### Maintain in acct_id record:
When a new account is created, add fraud detection criteria:
* Fraud criteria (30 percent over average threshold for payee, thresh = 30), can be tuned in future. (Do once when acct_id record is created.)

Upon incoming withdrawl transaction, atomically maintain:
* List of unique payees.
* Maintain average withdrawl amount for each payee.  
_(Maintain sum of all withdrawls, count of withdrawl. avg = sum/count)_ 
* **current_balance** if transaction is not rejected or flagged as fraudulent

We will internally maintain the following additional bins. Data will be updated with every incoming transaction, atomically, by using Expressions.

![fig_DM_3](./graphics/Fraud_DM_3.png)





In [51]:
import com.aerospike.client.cdt.MapPolicy;
import com.aerospike.client.cdt.MapOrder;
import com.aerospike.client.cdt.MapWriteFlags;
import com.aerospike.client.cdt.MapReturnType;
import com.aerospike.client.exp.MapExp;
import com.aerospike.client.cdt.CTX;
import java.util.HashMap;
import java.util.Map;
import com.aerospike.client.cdt.MapOperation;
import com.aerospike.client.policy.RecordExistsAction;

int iAcct_id = 3;  //Acct_id = 1 to 1000.
boolean bOvr = false;  //<-- Note: we set Overrride to false  (wrong)
int iWdAmount = -2000;
String sPayee = "SELF";
String sProfile = "user"+iAcct_id;  //Check we are not updating the profile string now.
int iThreshold = 30;

Key user = new Key("test", "accts", iAcct_id); 

Bin bOverride = new Bin("ovr", Value.get(bOvr));
Bin bAmnt = new Bin("lastamnt",Value.get(iWdAmount));
Bin bPayee = new Bin("payee",Value.get(sPayee));
Bin bProfile = new Bin("profile", Value.get(sProfile));
Bin bThreshold = new Bin("thresh", Value.get(iThreshold));

if(bOvr){
    wPolicy.recordExistsAction = RecordExistsAction.CREATE_ONLY;   //when creating acct_id record
}else {
    wPolicy.recordExistsAction = RecordExistsAction.UPDATE_ONLY;   //acct_id record must exist.
}
//User 3 account is not created yet. If we create with bOvr=false, we should get an error.

Expression thresholdExp = Exp.build(Exp.val(iThreshold));
    
MapPolicy mPolicy = new MapPolicy(MapOrder.KEY_ORDERED, MapWriteFlags.DEFAULT);

//If MapKey sPayee exists, add to sum and count, otherwise create sum and count entries for map key = sPayee
//Value returned by this expression will be written to sPayeee{sum:expValue}
Exp payeeSumExp = 
    // If ..
    Exp.cond( 
        Exp.eq( 
            // The number of records corresponding to key sPayee in the mapPayees bin
            MapExp.getByKey(MapReturnType.COUNT, Exp.Type.INT, Exp.val(sPayee), Exp.mapBin("mapPayees")), 
            // is equal to zero
            Exp.val(0)
        ), 
        // then the sum of payments to sPayee is equal to the current payment amount
        Exp.val(iWdAmount),
        // else if this count
        Exp.eq( 
            MapExp.getByKey(MapReturnType.COUNT, Exp.Type.INT, Exp.val(sPayee), Exp.mapBin("mapPayees")), 
            // is equal to one
            Exp.val(1)
        ), 
        // then get the currently stored sum of all payments for this payee
        Exp.add(
            MapExp.getByKey(MapReturnType.VALUE, Exp.Type.INT, 
                                  Exp.val("sum"), Exp.mapBin("mapPayees"), 
                                  CTX.mapKey(Value.get(sPayee))), 
            // and add to it the payment amount
            Exp.val(iWdAmount)
        ),
        // Finally if the answer is > 1 ( can't happen ) return 0
        Exp.val(0) //default
    );
    
// This expression saves the result of the payeeSumExp calculation
Expression savePayeeSumExp = 
    Exp.build( 
        MapExp.put(
            // As 'sum'
            mPolicy, Exp.val("sum"), payeeSumExp, 
            //In the mapPayees bin
            Exp.mapBin("mapPayees"), 
            // Under the payee key
            CTX.mapKey(Value.get(sPayee))
        )
    );

//Value returned by payeeCountExp will be written to sPayeee{count:expValue}
//The structure is very similar to sumCountExp
//Except that we set the count to 1 if the payee has not been seen before
//And increment the count if it has
Exp payeeCountExp = 
    // If ..    
    Exp.cond( 
        Exp.eq( 
            // The number of records corresponding to key sPayee in the mapPayees bin            
            MapExp.getByKey(MapReturnType.COUNT, Exp.Type.INT, Exp.val(sPayee), Exp.mapBin("mapPayees")), 
            // is equal to zero            
            Exp.val(0)), 
        // then return 1
        Exp.val(1),
        // else if this count        
        Exp.eq( 
              MapExp.getByKey(MapReturnType.COUNT, Exp.Type.INT, Exp.val(sPayee), Exp.mapBin("mapPayees")), 
        // is equal to one
              Exp.val(1)
        ), 
        // then get the currently stored count of all payments for this payee    
        Exp.add(
            MapExp.getByKey(MapReturnType.VALUE, Exp.Type.INT, 
                                Exp.val("count"), Exp.mapBin("mapPayees"), 
                                CTX.mapKey(Value.get(sPayee))), 
            // and add one to it
            Exp.val(1)
        ),
        // Finally if the answer is > 1 ( can't happen) return 0
        Exp.val(0) //default
    );
 
// Similarly this expression saves the result of the payeeCountExp calculation
Expression savePayeeCountExp = 
    Exp.build( 
        // as 'count'
        MapExp.put(
            mPolicy, Exp.val("count"), payeeCountExp, 
            //In the mapPayees bin
            Exp.mapBin("mapPayees"), 
            // Under the payee key
            CTX.mapKey(Value.get(sPayee))
        )
    );

// Need to initialise the payee map
Map<Value, Value> initPayeeMap = new HashMap<Value, Value>();
initPayeeMap.put(Value.get("sum"), Value.get(0));
initPayeeMap.put(Value.get("count"), Value.get(0));

// The policy we use will make sure the initalisation map is only saved if it doesn't exist
MapPolicy mPolicyInit = 
    new MapPolicy(
        MapOrder.KEY_ORDERED, 
        MapWriteFlags.CREATE_ONLY|MapWriteFlags.NO_FAIL|MapWriteFlags.PARTIAL
    );  

// Re-use wPolicy / balExp / profileExp
// In a try/catch as the filtered out error may be thrown
try {
    record = client.operate( 
              wPolicy, user,   
              Operation.put(bOverride),
              Operation.put(bAmnt),     
              Operation.put(bPayee),    
              // Previously seen balance expression
              ExpOperation.write("bal", balExp, ExpWriteFlags.DEFAULT),
              // Previously seen profile expression
              ExpOperation.write("profile", profileExp, ExpWriteFlags.CREATE_ONLY | ExpWriteFlags.POLICY_NO_FAIL),
              // Set the fraud detection threshold
              ExpOperation.write("thresh", thresholdExp, ExpWriteFlags.CREATE_ONLY | ExpWriteFlags.POLICY_NO_FAIL),
              // Initialise the payee map if required
              MapOperation.put(mPolicyInit, "mapPayees", Value.get(sPayee), Value.get(initPayeeMap)),
              // Save the sum of the amounts paid to this payee so far using the savePayeeSumExp expression
              ExpOperation.write("mapPayees", savePayeeSumExp, ExpWriteFlags.EVAL_NO_FAIL),
              // Similarly save teh sum of the count of payments to this payee
              ExpOperation.write("mapPayees", savePayeeCountExp, ExpWriteFlags.EVAL_NO_FAIL)
    );
} 
catch (AerospikeException e){
    int rc = e.getResultCode();
    if(rc == ResultCode.FILTERED_OUT){
        System.out.println("Insufficient Balance or Fraud. Transaction Rejected.");
    }
    else{
        System.out.println("Operate error. Resultcode: "+ ResultCode.getResultString(rc));     
    }    
}

In [53]:
//Check 
//Key user = new Key("test", "accts", 3); 
System.out.println( client.get(null, user) );

(gen:2),(exp:392914495),(bins:(ovr:0),(lastamnt:-2000),(payee:SELF),(bal:6000),(profile:do_not_update_user2),(thresh:30),(mapPayees:{SELF={count=2, sum=-4000}}))


#### Update Record Data Model for Fraud Detection related bins
* payeelist Map data type with key=payee name, value = [cumulative_withdawls, count] to compute average.  Estimate record size vs max number of payees. 


![fig_DM_3](./graphics/Fraud_DM_3.png)


* Decide record size to use 128K / 256K / 512K / 1M / 2M / 4M / 8M? 1M or below is preferred.
 *  **Sizing Estimate:**
 *  Use device storage sizing guideline to make an estimate using an average payee name string length.
 *  Should we hash the payee name to known number of bits to get consistent sizing? 
  *   CDTs allow byte arrays to be map keys.
  *   Stay with string payee name map key for today's exercise.
* Fraud threshold percentage bin: _thresh_ = 30. 


#### Flag as Potential_Fraud if override is false and:
* If _withdrawl_amount_ is greater than x (e.g.30)% of average. 
* Skip fraud check on first 5 transactions of a payee to build the average.
* Don't apply fraud checkn on Credit amounts (negative withdrawl)

* **If flagged as Potential_Fraud:** 
 *  do not add payee to unique list
 *  do not update payee average withdrawl amount.
 *  do not update transaction withdrawl_amount or current_balance.
 

In [68]:
import com.aerospike.client.cdt.MapPolicy;
import com.aerospike.client.cdt.MapOrder;
import com.aerospike.client.cdt.MapWriteFlags;
import com.aerospike.client.cdt.MapReturnType;
import com.aerospike.client.exp.MapExp;
import com.aerospike.client.cdt.CTX;
import java.util.HashMap;
import java.util.Map;
import com.aerospike.client.cdt.MapOperation;
import com.aerospike.client.policy.RecordExistsAction;

int iAcct_id = 3;  //Acct_id = 1 to 1000.
boolean bOvr = true;
int iWdAmount = -5000;
String sPayee = "SELF";
String sProfile = "user"+iAcct_id;  //Check we are not updating the profile string now.
int iThreshold = 30;

Key user = new Key("test", "accts", iAcct_id); 

Bin bOverride = new Bin("ovr", Value.get(bOvr));
Bin bAmnt = new Bin("lastamnt",Value.get(iWdAmount));
Bin bPayee = new Bin("payee",Value.get(sPayee));
Bin bProfile = new Bin("profile", Value.get(sProfile));
Bin bThreshold = new Bin("thresh", Value.get(iThreshold));

if(bOvr){
    wPolicy.recordExistsAction = RecordExistsAction.CREATE_ONLY;   //when creating acct_id record
}else {
    wPolicy.recordExistsAction = RecordExistsAction.UPDATE_ONLY;   //acct_id record must exist.
}

// Take the existing filter expression and add to it a fraud detecion filter
wPolicy.filterExp = Exp.build(
    Exp.and(
      Exp.or(
        Exp.cond(Exp.binExists("bal"), Exp.ge(Exp.intBin("bal"),Exp.val(iWdAmount)), Exp.val(false)),
        //Note: This filter will be ignore if the record does not exist
        //Hence, we use recordExistsAction policy per above based on bOvr value.
        Exp.val(bOvr)
      ),
      //Lets now add fraud detection filter- withdrawl amount should be within 30% of average withdrawl amount.
      //Then, if count >5, check withdrawl amount < or = (100+threshold)*(sum/count)/100
      Exp.cond(
          //First if withdrawl amount is negative (<0), return true          
          Exp.lt(Exp.val(iWdAmount),Exp.val(0)), Exp.val(true),
          //If mapPayees bin does not exist, return true, no transaction in yet.          
          Exp.not(Exp.binExists("mapPayees")), Exp.val(true),
          //If mapPayees does not have entry for sPayee, return true, no transaction in yet          
          Exp.eq(
              MapExp.getByKey(MapReturnType.COUNT, Exp.Type.INT, Exp.val(sPayee), 
                                      Exp.mapBin("mapPayees")), 
              Exp.val(0)), Exp.val(true),
          // But if the number of txns for this payee exceeds 5
          Exp.gt(
              // ... value of count key
              MapExp.getByKey(MapReturnType.VALUE, Exp.Type.INT, Exp.val("count"), 
                  // in the mapPayees bin, under the key sPayee
                  Exp.mapBin("mapPayees"),CTX.mapKey(Value.get(sPayee)))
              , Exp.val(5)
          ),
          // Check that
          Exp.le(
              // The amount is less than
              Exp.val(iWdAmount),
              Exp.div(
                  Exp.mul(
                      // Threshold + 100(%) *                      
                      Exp.add(Exp.val(100),Exp.intBin("thresh")), 
                      Exp.div(
                          // the sum of all payments to that payee                          
                          MapExp.getByKey(MapReturnType.VALUE, Exp.Type.INT, Exp.val("sum"), 
                                  Exp.mapBin("mapPayees"),CTX.mapKey(Value.get(sPayee))),
                          // divided by the count of the payments ot that payee
                          MapExp.getByKey(MapReturnType.VALUE, Exp.Type.INT, Exp.val("count"), 
                                  Exp.mapBin("mapPayees"),CTX.mapKey(Value.get(sPayee)))                
                     )
                  ), 
                  // divided by 100 as we're working in pct
                  Exp.val(100)
              )
          )
        // and if so, return true  
        ,Exp.val(true)
     )
    )
);
wPolicy.failOnFilteredOut = true;

// Re-use wPolicy / balExp / profileExp
// In a try/catch as the filtered out error may be thrown
try {
    record = client.operate( 
              wPolicy, user,   
              Operation.put(bOverride),
              Operation.put(bAmnt),     
              Operation.put(bPayee),    
              // Previously seen balance expression
              ExpOperation.write("bal", balExp, ExpWriteFlags.DEFAULT),
              // Previously seen profile expression
              ExpOperation.write("profile", profileExp, ExpWriteFlags.CREATE_ONLY | ExpWriteFlags.POLICY_NO_FAIL),
              // Set the fraud detection threshold
              ExpOperation.write("thresh", thresholdExp, ExpWriteFlags.CREATE_ONLY | ExpWriteFlags.POLICY_NO_FAIL),
              // Initialise the payee map if required
              MapOperation.put(mPolicyInit, "mapPayees", Value.get(sPayee), Value.get(initPayeeMap)),
              // Save the sum of the amounts paid to this payee so far using the savePayeeSumExp expression
              ExpOperation.write("mapPayees", savePayeeSumExp, ExpWriteFlags.EVAL_NO_FAIL),
              // Similarly save teh sum of the count of payments to this payee
              ExpOperation.write("mapPayees", savePayeeCountExp, ExpWriteFlags.EVAL_NO_FAIL)
    );
} 
catch (AerospikeException e){
    int rc = e.getResultCode();
    if(rc == ResultCode.FILTERED_OUT){
        System.out.println("Insufficient Balance or Fraud. Transaction Rejected.");
    }
    else{
        System.out.println("Operate error. Resultcode: "+ ResultCode.getResultString(rc));     
    }    
}

Operate error. Resultcode: Key already exists


In [69]:
//Inspect 
//Key user = new Key("test", "accts", 3); 
System.out.println( client.get(null, user) );

(gen:5),(exp:392916615),(bins:(ovr:0),(lastamnt:-5000),(payee:SELF),(bal:21000),(profile:do_not_update_user2),(thresh:30),(mapPayees:{SELF={count=5, sum=-19000}}))


## Using Java classes and methods
 
 Lets see with a simple example how we can use Java classes and methods in Jupyter Notebook. We can clean up our implementation as a class method where we can pass the transaction attributes as an argument.

### Defing a Class
 

In [70]:
//CLASS Cell
class myTest {
  public void foo (int val ) {
    System.out.println("Calling myTest:foo() to print integer:"+val);
  }
}

### Using a Class Method
 

In [71]:
myTest myobj = new myTest();
myobj.foo(5);

Calling myTest:foo() to print integer:5


## "Classify" our Model
 Let create a class _detectFraud_ with a method transact() that takes in a transaction with all its attributes.

In [84]:
//Add the imports. We already have them in our interactive kernel at this point.
//Classify 
class DetectFraud {
  public void transact(
    int iAcct_id,
    boolean bOvr,
    int iWdAmount,
    String sPayee,
    int iThreshold) {
 
String sProfile = "user"+iAcct_id;  //Check we are not updating the profile string now

Key user = new Key("test", "accts", iAcct_id); 

Bin bOverride = new Bin("ovr", Value.get(bOvr));
Bin bAmnt = new Bin("lastamnt",Value.get(iWdAmount));
Bin bPayee = new Bin("payee",Value.get(sPayee));
Bin bProfile = new Bin("profile", Value.get(sProfile));
Bin bThreshold = new Bin("thresh", Value.get(iThreshold));

WritePolicy wPolicy = new WritePolicy();

// Only way to create a record is with a -ve payment      
if(bOvr && (iWdAmount<0))
{
    wPolicy.recordExistsAction = RecordExistsAction.CREATE_ONLY;   //when creating acct_id record
}
else 
{
    wPolicy.recordExistsAction = RecordExistsAction.UPDATE_ONLY;   
}

// Take the existing filter expression and add to it a fraud detecion filter
wPolicy.filterExp = Exp.build(
    Exp.and(
      // Test to see we're not taking more out of the account than is in the account
      //Note: This filter will be ignored if the record does not exist        
      Exp.or(
        Exp.cond(Exp.binExists("bal"), Exp.ge(Exp.intBin("bal"),Exp.val(iWdAmount)), Exp.val(false)),
        //Hence, we use recordExistsAction policy per above based on bOvr value.
        Exp.val(bOvr)
      ),
      //Lets now add fraud detection filter- withdrawl amount should be within 30% of average withdrawl amount.
      //Then, if count >5, check withdrawl amount < or = (100+threshold)*(sum/count)/100
      Exp.cond(
          // Disable fraud detection if override is true
          Exp.val(bOvr), Exp.val(true),           
          //First if withdrawl amount is negative (<0), return true          
          Exp.lt(Exp.val(iWdAmount),Exp.val(0)), Exp.val(true),
          //If mapPayees bin does not exist, return true, no transaction in yet.          
          Exp.not(Exp.binExists("mapPayees")), Exp.val(true),
          //If mapPayees does not have entry for sPayee, return true, no transaction in yet          
          Exp.eq(
              MapExp.getByKey(MapReturnType.COUNT, Exp.Type.INT, Exp.val(sPayee), 
                                      Exp.mapBin("mapPayees")), 
              Exp.val(0)), Exp.val(true),
          // But if the number of txns for this payee exceeds 5
          Exp.gt(
              // ... value of count key
              MapExp.getByKey(MapReturnType.VALUE, Exp.Type.INT, Exp.val("count"), 
                  // in the mapPayees bin, under the key sPayee
                  Exp.mapBin("mapPayees"),CTX.mapKey(Value.get(sPayee)))
              , Exp.val(5)
          ),
          // Check that
          Exp.le(
              // The amount is less than
              Exp.val(iWdAmount),
              Exp.div(
                  Exp.mul(
                      // Threshold + 100(%) *                      
                      Exp.add(Exp.val(100),Exp.intBin("thresh")), 
                      Exp.div(
                          // the sum of all payments to that payee                          
                          MapExp.getByKey(MapReturnType.VALUE, Exp.Type.INT, Exp.val("sum"), 
                                  Exp.mapBin("mapPayees"),CTX.mapKey(Value.get(sPayee))),
                          // divided by the count of the payments ot that payee
                          MapExp.getByKey(MapReturnType.VALUE, Exp.Type.INT, Exp.val("count"), 
                                  Exp.mapBin("mapPayees"),CTX.mapKey(Value.get(sPayee)))                
                     )
                  ), 
                  // divided by 100 as we're working in pct
                  Exp.val(100)
              )
          )
        // and if so, return true  
        ,Exp.val(true)
     )
    )
);
wPolicy.failOnFilteredOut = true;

// Expression for calculating balance
Expression balExp = 
    Exp.build(
        // If balance exists
        Exp.cond(Exp.binExists("bal"), 
                 // Subtract the withdrawl amount from it
                 Exp.sub(Exp.intBin("bal"),Exp.val(iWdAmount)),
                 // Else subtract from zero
                 Exp.sub(Exp.val(0), Exp.val(iWdAmount))
        )
    );

// We will use the below to ensure that the profile only gets written once
Expression profileExp = Exp.build(Exp.val(sProfile));

Expression thresholdExp = Exp.build(Exp.val(iThreshold));
    
MapPolicy mPolicy = new MapPolicy(MapOrder.KEY_ORDERED, MapWriteFlags.DEFAULT);

//If MapKey sPayee exists, add to sum and count, otherwise create sum and count entries for map key = sPayee
//Value returned by this expression will be written to sPayeee{sum:expValue}
Exp payeeSumExp = 
    // If ..
    Exp.cond( 
        Exp.eq( 
            // The number of records corresponding to key sPayee in the mapPayees bin
            MapExp.getByKey(MapReturnType.COUNT, Exp.Type.INT, Exp.val(sPayee), Exp.mapBin("mapPayees")), 
            // is equal to zero
            Exp.val(0)
        ), 
        // then the sum of payments to sPayee is equal to the current payment amount
        Exp.val(iWdAmount),
        // else if this count
        Exp.eq( 
            MapExp.getByKey(MapReturnType.COUNT, Exp.Type.INT, Exp.val(sPayee), Exp.mapBin("mapPayees")), 
            // is equal to one
            Exp.val(1)
        ), 
        // then get the currently stored sum of all payments for this payee
        Exp.add(
            MapExp.getByKey(MapReturnType.VALUE, Exp.Type.INT, 
                                  Exp.val("sum"), Exp.mapBin("mapPayees"), 
                                  CTX.mapKey(Value.get(sPayee))), 
            // and add to it the payment amount
            Exp.val(iWdAmount)
        ),
        // Finally if the answer is > 1 ( can't happen ) return 0
        Exp.val(0) //default
    );
    
// This expression saves the result of the payeeSumExp calculation
Expression savePayeeSumExp = 
    Exp.build( 
        MapExp.put(
            // As 'sum'
            mPolicy, Exp.val("sum"), payeeSumExp, 
            //In the mapPayees bin
            Exp.mapBin("mapPayees"), 
            // Under the payee key
            CTX.mapKey(Value.get(sPayee))
        )
    );

//Value returned by payeeCountExp will be written to sPayeee{count:expValue}
//The structure is very similar to sumCountExp
//Except that we set the count to 1 if the payee has not been seen before
//And increment the count if it has
Exp payeeCountExp = 
    // If ..    
    Exp.cond( 
        Exp.eq( 
            // The number of records corresponding to key sPayee in the mapPayees bin            
            MapExp.getByKey(MapReturnType.COUNT, Exp.Type.INT, Exp.val(sPayee), Exp.mapBin("mapPayees")), 
            // is equal to zero            
            Exp.val(0)), 
        // then return 1
        Exp.val(1),
        // else if this count        
        Exp.eq( 
              MapExp.getByKey(MapReturnType.COUNT, Exp.Type.INT, Exp.val(sPayee), Exp.mapBin("mapPayees")), 
        // is equal to one
              Exp.val(1)
        ), 
        // then get the currently stored count of all payments for this payee    
        Exp.add(
            MapExp.getByKey(MapReturnType.VALUE, Exp.Type.INT, 
                                Exp.val("count"), Exp.mapBin("mapPayees"), 
                                CTX.mapKey(Value.get(sPayee))), 
            // and add one to it
            Exp.val(1)
        ),
        // Finally if the answer is > 1 ( can't happen) return 0
        Exp.val(0) //default
    );
 
// Similarly this expression saves the result of the payeeCountExp calculation
Expression savePayeeCountExp = 
    Exp.build( 
        // as 'count'
        MapExp.put(
            mPolicy, Exp.val("count"), payeeCountExp, 
            //In the mapPayees bin
            Exp.mapBin("mapPayees"), 
            // Under the payee key
            CTX.mapKey(Value.get(sPayee))
        )
    );

// Need to initialise the payee map
Map<Value, Value> initPayeeMap = new HashMap<Value, Value>();
initPayeeMap.put(Value.get("sum"), Value.get(0));
initPayeeMap.put(Value.get("count"), Value.get(0));

// The policy we use will make sure the initalisation map is only saved if it doesn't exist
MapPolicy mPolicyInit = 
    new MapPolicy(
        MapOrder.KEY_ORDERED, 
        MapWriteFlags.CREATE_ONLY|MapWriteFlags.NO_FAIL|MapWriteFlags.PARTIAL
    );  

// Re-use wPolicy / balExp / profileExp
// In a try/catch as the filtered out error may be thrown
try {
    record = client.operate( 
              wPolicy, user,   
              Operation.put(bOverride),
              Operation.put(bAmnt),     
              Operation.put(bPayee),    
              // Previously seen balance expression
              ExpOperation.write("bal", balExp, ExpWriteFlags.DEFAULT),
              // Previously seen profile expression
              ExpOperation.write("profile", profileExp, ExpWriteFlags.CREATE_ONLY | ExpWriteFlags.POLICY_NO_FAIL),
              // Set the fraud detection threshold
              ExpOperation.write("thresh", thresholdExp, ExpWriteFlags.CREATE_ONLY | ExpWriteFlags.POLICY_NO_FAIL),
              // Initialise the payee map if required
              MapOperation.put(mPolicyInit, "mapPayees", Value.get(sPayee), Value.get(initPayeeMap)),
              // Save the sum of the amounts paid to this payee so far using the savePayeeSumExp expression
              ExpOperation.write("mapPayees", savePayeeSumExp, ExpWriteFlags.EVAL_NO_FAIL),
              // Similarly save teh sum of the count of payments to this payee
              ExpOperation.write("mapPayees", savePayeeCountExp, ExpWriteFlags.EVAL_NO_FAIL)
    );
} 
catch (AerospikeException e){
    int rc = e.getResultCode();
    if(rc == ResultCode.FILTERED_OUT){
 System.out.println("Insufficient Balance or Fraud. Transaction Rejected. Payee: "+sPayee+" Amount: " + iWdAmount);    }
    else{
        System.out.println("Operate error. Resultcode: "+ ResultCode.getResultString(rc));     
    }    
}
}}

## Test our "Classified" Model
 

### Restart clean on Aerospike server
 

In [85]:
%sh aql -h 127.0.0.1 -f "./aqlScripts/truncate_test.aql"

In [86]:
DetectFraud myFraudDetector = new DetectFraud();
myFraudDetector.transact(1,false,1000,"VISA",30);
myFraudDetector.transact(1,true,-5000,"SELF",30);
myFraudDetector.transact(1,false,-10000,"SELF",30);
myFraudDetector.transact(1,false,1000,"VISA",30);
myFraudDetector.transact(1,false,1500,"Costco",30);
myFraudDetector.transact(1,false,500,"VISA",30);
myFraudDetector.transact(1,false,1000,"VISA",30);
myFraudDetector.transact(1,false,1500,"Costco",30);
myFraudDetector.transact(1,false,-5000,"SELF",30);
myFraudDetector.transact(1,false,1000,"VISA",30);
myFraudDetector.transact(1,false,2000,"Costco",30);
myFraudDetector.transact(1,false,1500,"VISA",30);
myFraudDetector.transact(1,false,1000,"VISA",30);
myFraudDetector.transact(1,false,3000,"VISA",30);
myFraudDetector.transact(1,false,1000,"VISA",30);

Operate error. Resultcode: Key not found
Insufficient Balance or Fraud. Transaction Rejected. Payee: VISA Amount: 3000


In [87]:
//Check 
Key user = new Key("test", "accts", 1); 
System.out.println( client.get(null, user) );

(gen:13),(exp:392918384),(bins:(ovr:0),(lastamnt:1000),(payee:VISA),(bal:8000),(profile:user1),(thresh:30),(mapPayees:{Costco={count=3, sum=5000}, SELF={count=3, sum=-20000}, VISA={count=7, sum=7000}}))


### Try other transactions
 

### Clean-up Aerospike server

We will populate with a larger dataset.
 

In [95]:
%sh aql -h 127.0.0.1 -f "./aqlScripts/truncate_test.aql"

## Insert Generated Random Test Data
Let's create 1000 acct_ids, and then insert ~100K transactions for 1000 acct_ids [0001 thru 1000], average 10 transactions per acct_id per payee, each acct_id has ~10 unique payees. (Average 100 transactions per payee per acct_id).

1. Create acct_ids
2. Upload/insert transacitons. (Underlying code should build needed fraud detection bins' data with every transaction.)
3. Transactions include deposit transactions also.
4. All these inserts are done with override=true.

**Payee names in Test Data:** VISA, CitiMortgage, Costco, HOA, Joe_Landscaper, PacificElectric, CityWater, Jane_Helper, John_Doe, Cash. (14 characters max here.)


In [96]:
import java.util.ArrayList;
import java.util.List;
import java.util.Random;

class GenerateTestData {

    public void generate(){
        DetectFraud myFraudDetector = new DetectFraud();

        //Create acct_id with initial balance of 10000
        for(int i=1;i<1001;i++){
          myFraudDetector.transact(i,true,-10000,"SELF",30);
        }

        String[] payee = new String[10];
        payee[0]="VISA";
        payee[1]="CitiMortgage";
        payee[2]="Costco";
        payee[3]="HOA";
        payee[4]="Joe_Landscaper";
        payee[5]="PacificElectric";
        payee[6]="CityWater";
        payee[7]="Jane_Helper";
        payee[8]="John_Doe";
        payee[9]="Cash";

        Random rnd1 = new Random(0);  
        //rnd1.nextInt(20) will give a random number between 0(incl.) and 20(excl.)

        Random rnd2 = new Random(0);

        for(int i=1;i<1001;i++){
          int rndLimit = rnd1.nextInt(20);  //Transaction loops for this acct_id
          for(int j=0;j<rndLimit;j++){
            for(int k=0;k<10;k++){  //10 unique payees
               myFraudDetector.transact(i,true,(200+rnd2.nextInt(200)),payee[k],30);
               //Override is true - no fraud detection enabled.
            } 
            myFraudDetector.transact(i,false,-4000,"SELF",30);  //Replenish balance
          }  
        }

    }
}

GenerateTestData genData = new GenerateTestData();
genData.generate();

## Test the Fraud Model
Insert a fraudulent transaction. Typically, it will be a payee withdrawl as "Cash" with a suspicious amount.


In [109]:
//Check any user
Key user = new Key("test", "accts", 5); 
System.out.println( client.get(null, user) );

(gen:173),(exp:392918800),(bins:(ovr:0),(lastamnt:449),(payee:Cash),(bal:22880),(profile:user5),(thresh:50),(mapPayees:{Cash={count=20, sum=6495}, CitiMortgage={count=15, sum=4711}, CityWater={count=15, sum=4590}, Costco={count=15, sum=4414}, HOA={count=15, sum=4472}, Jane_Helper={count=15, sum=4767}, Joe_Landscaper={count=15, sum=4620}, John_Doe={count=15, sum=4259}, PacificElectric={count=15, sum=4659}, SELF={count=16, sum=-70000}, VISA={count=15, sum=4133}}))


### Test for this user
Insert a fraudulent transaction. Typically, it will be a payee withdrawl as "Cash" with a suspicious amount.


In [112]:
DetectFraud myFraudDetector = new DetectFraud();
myFraudDetector.transact(5,false,486,"Cash",30);

In [101]:
//Check last transaction (if it succeeds)
Key user = new Key("test", "accts", 5); 
System.out.println( client.get(null, user) );

(gen:167),(exp:392918658),(bins:(ovr:0),(lastamnt:379),(payee:Cash),(bal:24606),(profile:user5),(thresh:30),(mapPayees:{Cash={count=16, sum=4769}, CitiMortgage={count=15, sum=4711}, CityWater={count=15, sum=4590}, Costco={count=15, sum=4414}, HOA={count=15, sum=4472}, Jane_Helper={count=15, sum=4767}, Joe_Landscaper={count=15, sum=4620}, John_Doe={count=15, sum=4259}, PacificElectric={count=15, sum=4659}, SELF={count=16, sum=-70000}, VISA={count=15, sum=4133}}))


## Model Tuning
 Have a way to update the fraud detection criteria, in our example, value in the threshold percentage bin (_thresh_). 
 Scan all records (we have 1000) and update _threshold percentage_ value in each record.
 
 
![fig_DM_3](./graphics/Fraud_DM_3.png)



In [103]:
import com.aerospike.client.task.ExecuteTask;
import com.aerospike.client.query.Statement;

Statement stmt = new Statement();
stmt.setNamespace("test");
stmt.setSetName("accts");

//Use scan execute() with expressions.
Expression exp = Exp.build(Exp.val(50)); //Change threshold to 50%
Operation[] operations = new Operation[1];
operations[0]= ExpOperation.write("thresh", exp, ExpWriteFlags.DEFAULT);
stmt.setOperations(operations);

WritePolicy wPolicyScan = new WritePolicy();
ExecuteTask et = client.execute(wPolicyScan, stmt)

## Retest
Hop back to previous cells, for a particular user, check new threshold, test. Calculate fraud threshold from record data, insert a fraudulent transaction and check. Insert a good transaction and check.

# Epilogue
We have assumed an extremely simple example to deliver a hands-on experience in the limited amount of time on our hands. The goal was to demonstrate some of the capabilities and features available with Aerospike. While experts in fraud detection will develop much more sophisticated models, we hope they find these techinques helpful.

# But wait, there is more :-)
We also have the Aerospike Monitoring Stack running on our single AWS Instance.

![fig_setup_2](./graphics/fraud_det_setup.png)

Lets take a quick look at the Grafana Dashboard in the browser. 

Go to your instance's  Public IP address and port 8000. 

### http://your_public_ip_addr:8000 

### Login Id: admin

### Password: admin

### SKIP Changing the password.

![fig_gf_1](./graphics/grafana_login.png)

### Load _Cluster Overview_ and _Namespace View_ Dashboards from _Aerospike_ folder


![fig_gf_2](./graphics/grafana_dashboard_load.png)

### Cluster Overview Dashboard

Shows information about the Aerospike Cluster

![fig_gf_3](./graphics/cluster_view.png)

### Namespace View Dashboard 

Should show 1K Master Objects

![fig_gf_4](./graphics/namespace_view.png)





# But wait, I can't Deploy Everything on a Single Machine in Practice


# How can I quickly Deploy something like this ...


![fig_qs_1](./graphics/aws_quickstart.png)


## Yes you can - with  AWS Quickstart  - Lets Explore that next.


